In [1]:
# author: Yuchen Song
# student_id: 201830360498
import numpy as np
import os
import nltk
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Bidirectional, Input, Dense, Activation, Embedding, Dropout, TimeDistributed, GRU, Add, Lambda
from tensorflow.keras.layers import dot, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback
from data_helper import load_data

os.environ['CUDA_VISIBLE_DEVICES']="" # uncomment this line, if you use cpu

In [2]:
# training parameters
dropout_rate = 0.2
epochs = 20
batch_size = 64
embedding_dim = 100

# gru parameters
hidden_dim = 100
num_encoder_layer = 2
num_decoder_layer = 1

# attention parameters
# None donates that no self-attenti0n mechanism is used
# 'dot' indicated the self-attention mechanism presented in the tutorial
attention_type = 'dot'  # [None, 'dot', 'multiplicative', 'additive']


def seq2seq_predict(seq2seq_model, encoder_input, decoder_sequence_length, sos_idx):
    # because we do not have the truth decoder input,
    # we need to use the decoder prediction as its input
    decoder_input = np.zeros(
        shape=(len(encoder_input), decoder_sequence_length))
    decoder_input[:, 0] = sos_idx
    for i in range(1, decoder_sequence_length):
        output = seq2seq_model.predict(
            [encoder_input, decoder_input], batch_size=batch_size).argmax(axis=2)
        decoder_input[:, i] = output[:, i-1]
    return output


def recover_sentence(x, idx2word):
    s = []
    for idx in x:
        word = idx2word[idx]
        if word == '<sos>':
            continue
        elif word == '<eos>':
            break
        elif word == '<pad>':
            break
        s.append(word)
    return s


class TestCallback(Callback):
    """
    Calculate BLEU
    """

    def __init__(self, test_data, model, vocabulary):
        self.test_data = test_data
        self.model = model
        self.vocabulary = vocabulary
        self.idx2word = dict()
        for k, v in self.vocabulary.items():
            self.idx2word[v] = k

    def on_epoch_end(self, epoch, logs={}):
        [encoder_input, decoder_input, decoder_target] = self.test_data
        decoder_output = seq2seq_predict(
            self.model, encoder_input, decoder_input_train.shape[1], vocabulary['<sos>'])
        bleu, results = self.evaluate_bleu(decoder_target, decoder_output)
        results.sort(reverse=True)
        print('Validation Set BLEU: %f' % (bleu))
        print('Top | BLEU | %s | %s' %
              ('target'.ljust(20), 'output'.ljust(20)))
        indices = list(range(len(results)))
        candidate_indices = list()
        candidate_indices.extend(indices[0:3])
        step = len(indices)//10
        if step > 0:
            candidate_indices.extend(indices[2+step::step])
        if indices[-1] != candidate_indices[-1]:
            candidate_indices.append(indices[-1])
        for i in candidate_indices:
            r = results[i]
            print('%-4d|%.4f| %s | %s' %
                  (i, r[0], ' '.join(r[1]), ' '.join(r[2])))

    def evaluate_bleu(self, target, output):
        N = target.shape[0]
        sum_bleu = 0.0
        results = []
        for i in range(N):
            t = recover_sentence(target[i], self.idx2word)
            o = recover_sentence(output[i], self.idx2word)
            bleu = nltk.translate.bleu_score.sentence_bleu([t], o)
            sum_bleu += bleu
            results.append((bleu, t, o))
        return sum_bleu / N, results

In [3]:
if __name__ == '__main__':
    print('Loading data')
    encoder_input_train, decoder_input_train, decoder_target_train, \
        encoder_input_valid, decoder_input_valid, decoder_target_valid, vocabulary = load_data(
            'translation')
    vocab_size = len(vocabulary)

    print('encoder_input_train.shape', encoder_input_train.shape) # （12244，10）
    print('decoder_input_train.shape', decoder_input_train.shape) # （12244，10）
    print('Vocab Size', vocab_size) # 3243

    num_training_data = encoder_input_train.shape[0]
    encoder_sequence_length = encoder_input_train.shape[1]
    decoder_sequence_length = decoder_input_train.shape[1]

    # encoder_input -> [batch_size, encoder_sequence_length]
    # decoder_input -> [batch_size, decoder_sequence_length]
    encoder_input = Input(shape=(encoder_sequence_length,), dtype='int32')
    decoder_input = Input(shape=(decoder_sequence_length,), dtype='int32')

    # the encoder and decoder share the same embedding layer
    emb_layer = Embedding(input_dim=vocab_size,
                          output_dim=embedding_dim, mask_zero=True)

Loading data
encoder_input_train.shape (12244, 10)
decoder_input_train.shape (12244, 10)
Vocab Size 3243


In [5]:
    ################
    # ENCODER PART #
    ################

    # embedding -> [batch_size, sequence_length, embedding_dim]
    ### YOUR CODE HERE ###
    encoder_input_embed = emb_layer(encoder_input)

    # dropout at embedding layer
    ### YOUR CODE HERE ###
    encoder_input_droped = Dropout(dropout_rate)(encoder_input_embed)

    # add multiple Bidirectional GRU layers here,
    # set units=hidden_dim, return_sequences=True at the previous layers
    # set units=hidden_dim, return_sequences=True, return_state=True at the last layer
    # please read https://keras.io/layers/recurrent/
    # output:
    #     if return_sequences==True:
    #         gru_output -> [batch_size, sequence_length, 2*hidden_dim]
    #     if return_sequences==True and return_state=True:
    #         gru_output -> [batch_size, sequence_length, 2*hidden_dim], [batch_size, hidden_dim], [batch_size, hidden_dim]

    encoder_inputs = [encoder_input_droped]
    for i in range(0, num_encoder_layer-1):
        # N − 1 layer(s) of Bidirectional GRU, which return(s) sequences only, i.e., set return_sequences=True, unroll=True.
        ### YOUR CODE HERE ###
        encoder_output = Bidirectional(GRU(hidden_dim, return_sequences=True, unroll=True))(encoder_inputs[i])
        # Dropout layers between GRU layers if applicable.
        ### YOUR CODE HERE ###
        encoder_output_droped = Dropout(dropout_rate)(encoder_output)

        encoder_inputs.append(encoder_output_droped)
    # 1 layer of Bidirectional GRU, which returns sequences and the last state.
    encoder_output, encoder_last_h, encoder_last_hr = Bidirectional(GRU(units=hidden_dim,
                                                                        return_sequences=True, 
                                                                        return_state=True, 
                                                                        unroll=True))(encoder_inputs[-1])

In [6]:
    ################
    # DECODER PART #
    ################

    # embedding -> [batch_size, sequence_length, embedding_dim]
    ### YOUR CODE HERE ###
    decoder_input_embed = emb_layer(decoder_input)

    # dropout at embedding layer
    ### YOUR CODE HERE ###
    decoder_input_droped = Dropout(dropout_rate)(decoder_input_embed)

    # add multiple Unidirectional GRU layers here,
    # set units=2*hidden_dim, return_sequences=True
    # set initial_state=encoder_hidden_state at the first layer
    # please read https://keras.io/layers/recurrent/
    # output:
    # gru_output -> [batch_size, sequence_length, 2*hidden_dim]

    # 1 layer of Unidirectional GRU, whose input is the output of the encoder’s last layer
    # and initial hidden state is the encoder’s last state, i.e., initial_state=concatenate([encoder_last_h, encoder_last_hr], axis=1)
    ### YOUR CODE HERE ###
    initial_state = concatenate([encoder_last_h, encoder_last_hr], axis=1)
    decoder_output = GRU(2*hidden_dim, return_sequences=True, unroll=True)(decoder_input_droped, initial_state)
    decoder_outputs = [decoder_output]
    # M − 1 layer(s) of Unidirectional GRU.
    # Dropout layers between GRU layers if applicable (optional).
    for i in range(1, num_decoder_layer):
        ### YOUR CODE HERE ###
        decoder_outputs[i] = Dropout(dropout_rate)(decoder_outputs[i])
        pass

    # simple seq2seq without any attention mechanism
    if attention_type is None:
        # 1 layer of Dense layer with softmax activation wrapped by TimeDistributed.
        output = TimeDistributed(
            Dense(units=vocab_size, activation='softmax'))(decoder_outputs[-1])
    else:
        # dot-product attention
        # weight_{i,j} = softmax(\sum_k {decoder_output_{i,k} * encoder_output_{j,k}})
        if attention_type == 'dot':
            ### YOUR CODE HERE ###
            weight = Activation('softmax')(
                dot([decoder_outputs[-1], encoder_output], axes=[2, 2]))
        # multiplicative attention
        # weight_{i,j} = softmax(\sum_k {decoder_output_{i,k} * (W encoder_output_{j,k})})
        elif attention_type == 'multiplicative':
            encoder_output_maped = Dense(units=2*hidden_dim, bias=False)(encoder_output)
            weight = Activation('softmax')(
                dot([decoder_outputs[-1], encoder_output_maped], axes=[2, 2]))
        # additive attention
        # weight_{i, j} = softmax(\sum_k {V tanh(W1 decoder_output_{i,k} + W2 encoder_output_{j,k})})
        elif attention_type == 'additive':
            # You may need the help of the Lambda wrapper(https://keras.io/layers/core/#lambda)
            # the K.expand_dims function and the K.squeeze function(https://keras.io/backend/#backend-functions
            decoder_output_maped = Dense(units=2*hidden_dim, bias=False)(decoder_output)
            encoder_output_maped = Dense(units=2*hidden_dim, bias=False)(encoder_output)
            tanh_result = Activation('tanh')(Add()([
                Lambda(lambda x: K.expand_dims(x, axis=2))(decoder_output_maped), 
                Lambda(lambda x: K.expand_dims(x, axis=1))(encoder_output_maped)]))
            weight = Activation('softmax')(
                Lambda(lambda x: K.squeeze(x, axis=3))(
                    Dense(units=1, bias=False)(tanh_result)))
        else:
            raise NotImplementedError
        attention = dot([weight, encoder_output], axes=[2, 1])
        output = TimeDistributed(Dense(units=vocab_size, activation='softmax'))(
            concatenate([decoder_outputs[-1], attention], axis=2))

    model = Model(inputs=[encoder_input, decoder_input], outputs=[output])

    adam = Adam()
    model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)

    print(model.summary())

    print("Traning Model...")
    history = model.fit([encoder_input_train, decoder_input_train], np.expand_dims(decoder_target_train, axis=2), 
                        batch_size=batch_size, 
                        epochs=epochs,
                        verbose=1 if os.name == 'posix' else 2, 
                        callbacks=[TestCallback((encoder_input_valid, decoder_input_valid, decoder_target_valid), model, vocabulary)])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 10)]         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 10)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 10, 100)      324300      input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
dropout_2 (Dropout)             (None, 10, 100)      0           embedding[1][0]              

G:\ANACODA3\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
G:\ANACODA3\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
G:\ANACODA3\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Validation Set BLEU: 0.002105
Top | BLEU | target               | output              
0   |1.0000| you re <unk> . | you re <unk> .
1   |1.0000| he s a <unk> . | he s a <unk> .
2   |0.5789| he s a <unk> liar . | he s a <unk> .
124 |0.0000| i m not very good at french . | i m not to to you .
246 |0.0000| i m done with you . | i m not to to you .
368 |0.0000| i m still busy . | i m very .
490 |0.0000| i m by your side . | i m not to .
612 |0.0000| you re alone aren t you ? | you re not to to .
734 |0.0000| i m going to start over . | i m very .
856 |0.0000| we re going the wrong way . | you re a to to <unk> .
978 |0.0000| you re not thinking clearly . | i m not to .
1100|0.0000| she speaks spanish well . | he s a .
1222|0.0000| you re one of them aren t you ? | i m not to to <unk> .
1224|0.0000| you are tired aren t you ? | i m not to to <unk> .
Epoch 2/20
192/192 - 10s - loss: 1.7743
Validation Set BLEU: 0.023647
Top | BLEU | target               | output              
0   |1.0000| we r

192/192 - 10s - loss: 0.6211
Validation Set BLEU: 0.339618
Top | BLEU | target               | output              
0   |1.0000| you re young . | you re young .
1   |1.0000| you re worried aren t you ? | you re worried aren t you ?
2   |1.0000| you re weak . | you re weak .
124 |1.0000| we re not <unk> . | we re not <unk> .
246 |1.0000| i m not in good shape now . | i m not in good shape now .
368 |0.6435| he is a very careful driver . | he is a very careful writer .
490 |0.3247| we are faced with a host of problems . | we re faced with the host of problems . .
612 |0.0000| i m not taking <unk> . | i m not going to <unk> .
734 |0.0000| he is afraid that he will die . | he is afraid of death .
856 |0.0000| we re already old . | we re old .
978 |0.0000| i m by your side . | i m your boss .
1100|0.0000| i m going to miss you a lot . | i m too tired of tired .
1222|0.0000| he s gone into hiding . | he is <unk> .
1224|0.0000| they are willing to help us out . | they re all mine .
Epoch 11/2

192/192 - 11s - loss: 0.3029
Validation Set BLEU: 0.613446
Top | BLEU | target               | output              
0   |1.0000| you re worried aren t you ? | you re worried aren t you ?
1   |1.0000| you re welcome . | you re welcome .
2   |1.0000| you re weak . | you re weak .
124 |1.0000| you re good . | you re good .
246 |1.0000| we re <unk> . | we re <unk> .
368 |1.0000| i m sure that s wrong . | i m sure that s wrong .
490 |1.0000| i m having trouble focusing . | i m having trouble focusing .
612 |1.0000| he is stronger than ever . | he is stronger than ever .
734 |0.5774| he s a <unk> . | he s a <unk> <unk> .
856 |0.0000| i m not joking . | i m not kidding .
978 |0.0000| i m totally drunk . | i m completely drunk .
1100|0.0000| we re still getting to know each other . | we re always trying to know anything .
1222|0.0000| they re terrified of you . | you re <unk> .
1224|0.0000| you are trusted by every one of us . | you re <unk> <unk> .
Epoch 20/20
192/192 - 11s - loss: 0.2847
Val